<a href="https://colab.research.google.com/github/ravi161995/Sentiment_Analyesis_project_01/blob/main/src/Notebooks/Copy_of_rnn_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from sklearn.metrics import classification_report

In [ ]:
train_path = "/content/drive/MyDrive/sentiment-cleaned-datasetskazanovasentiment140/train.csv"
test_path = "/content/drive/MyDrive/sentiment-cleaned-datasetskazanovasentiment140/test.csv"


In [ ]:
train = pd.read_csv(train_path, encoding='latin-1')
test = pd.read_csv(test_path, encoding='latin-1')

train.dropna(inplace=True)
test.dropna(inplace=True)

In [ ]:
X_train = train["cleaned_text"].values
y_train = train["target"].values
X_test  = test["cleaned_text"].values
y_test  = test["target"].values

In [ ]:
vocab_size = 5000
max_len = 50

tokenizer = Tokenizer(num_words=vocab_size, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq  = tokenizer.texts_to_sequences(X_test)

X_train_pad = pad_sequences(X_train_seq, maxlen=max_len, padding='post', truncating='post')
X_test_pad  = pad_sequences(X_test_seq, maxlen=max_len, padding='post', truncating='post')

In [ ]:
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

y_train shape: (1279740,)
y_test shape: (319938,)


In [ ]:
print("Vocab size:", vocab_size)
print("Max sequence length:", max_len)

Vocab size: 5000
Max sequence length: 50


In [ ]:
model = Sequential([
        Embedding(input_dim=vocab_size, output_dim=64, input_length=max_len),
        LSTM(64, dropout=0.2, recurrent_dropout=0.2),
        Dense(1, activation='sigmoid')
    ])

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:
model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])


model.build(input_shape=(None, max_len))
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 50, 64)         │       320,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 64)             │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 353,089 (1.35 MB)

 Trainable params: 353,089 (1.35 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
X_train_pad.shape
y_train.shape

(1279740,)

In [ ]:
history = model.fit(
        X_train_pad, y_train,
        validation_data=(X_test_pad, y_test),
        epochs=3,
        batch_size=64,
        verbose=1
    )

Epoch 1/3
19996/19996 ━━━━━━━━━━━━━━━━━━━━ 3394s 169ms/step - accuracy: 0.5000 - loss: 0.6933 - val_accuracy: 0.4984 - val_loss: 0.6932
Epoch 2/3
19996/19996 ━━━━━━━━━━━━━━━━━━━━ 3414s 171ms/step - accuracy: 0.5004 - loss: 0.6932 - val_accuracy: 0.4984 - val_loss: 0.6932
Epoch 3/3
19996/19996 ━━━━━━━━━━━━━━━━━━━━ 3430s 172ms/step - accuracy: 0.4994 - loss: 0.6932 - val_accuracy: 0.5016 - val_loss: 0.6932


In [ ]:
model.save('/content/drive/MyDrive/sentiment_model.h5')
print("✅ Model saved successfully to Drive!")


✅ Model saved successfully to Drive!


In [ ]:
y_pred = np.argmax(model.predict(X_test_pad), axis=1)
print(classification_report(y_test, y_pred, target_names=["Negative","Positive"]))

9999/9999 ━━━━━━━━━━━━━━━━━━━━ 318s 32ms/step
              precision    recall  f1-score   support

    Negative       0.50      1.00      0.67    159460
    Positive       0.00      0.00      0.00    160478

    accuracy                           0.50    319938
   macro avg       0.25      0.50      0.33    319938
weighted avg       0.25      0.50      0.33    319938



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
model.save('/content/drive/MyDrive/sentiment_model.h5')
print("✅ Model saved successfully to Drive!")

✅ Model saved successfully to Drive!
